# Inference

In [2]:
# imports
import ast
import os
import pathlib

import cv2
import neptune
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn_tutorial.datasets import ObjectDetectionDatasetSingle, ObjectDetectionDataSet
from pytorch_faster_rcnn_tutorial.faster_RCNN import get_faster_rcnn_resnet
from pytorch_faster_rcnn_tutorial.transformations import ComposeDouble
from pytorch_faster_rcnn_tutorial.transformations import ComposeSingle
from pytorch_faster_rcnn_tutorial.transformations import FunctionWrapperDouble
from pytorch_faster_rcnn_tutorial.transformations import FunctionWrapperSingle
from pytorch_faster_rcnn_tutorial.transformations import apply_nms, apply_score_threshold
from pytorch_faster_rcnn_tutorial.transformations import normalize_01
from pytorch_faster_rcnn_tutorial.utils import get_filenames_of_path, collate_single, save_json
from pytorch_faster_rcnn_tutorial.visual import DatasetViewer
from pytorch_faster_rcnn_tutorial.visual import DatasetViewerSingle
from pytorch_faster_rcnn_tutorial.backbone_resnet import ResNetBackbones

c:\Users\iserg\anaconda3\envs\Torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# parameters
params = {'EXPERIMENT': 'FRCNN-166',  # experiment name, e.g. Head-42
          'OWNER': 'svf',  # e.g. johndoe55
          'INPUT_DIR': 'pytorch_faster_rcnn_tutorial/data/stop_line/test_video',  # files to predict
          # 'INPUT_DIR': 'pytorch_faster_rcnn_tutorial/data/heads/test',  # files to predict
          'PREDICTIONS_PATH': 'pytorch_faster_rcnn_tutorial/data/stop_line/test_video/predictions',  # where to save the predictions
          'MODEL_DIR': 'pytorch_faster_rcnn_tutorial\data\stop_line\epoch=171-step=8771.ckpt',  # load model from checkpoint !epoch=275!306!
          'DOWNLOAD': False,  # whether to download from neptune
          'DOWNLOAD_PATH': 'model',  # where to save the model if DOWNLOAD is True
          'PROJECT': 'F-RCNN',  # Project name
          }

In [4]:
# input files
inputs = get_filenames_of_path(pathlib.Path(params['INPUT_DIR']))
inputs.sort()

In [5]:
# transformations
transforms = ComposeSingle([
    FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
    FunctionWrapperSingle(normalize_01)
])

In [6]:
# create dataset
dataset = ObjectDetectionDatasetSingle(inputs=inputs,
                                       transform=transforms,
                                       use_cache=False,
                                       )

In [7]:
# create dataloader
dataloader_prediction = DataLoader(dataset=dataset,
                                   batch_size=8,
                                   shuffle=False,
                                   num_workers=10,
                                   collate_fn=collate_single)

In [8]:
# api_key = os.environ['NEPTUNE']  # if this throws an error, you probably didn't set your env var
api_key = os.environ.get('NEPTUNE')

In [9]:
#api_key = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3MjYwMmIwNy1jMDZlLTRiYTYtODQyZC0zNGRkMmRjYzg2MmUifQ=="

In [10]:
# import experiment from neptune
project_name = f'{params["OWNER"]}/{params["PROJECT"]}'
project = neptune.init(project_qualified_name=project_name, api_token=api_key)  # get project
experiment_id = params['EXPERIMENT']  # experiment id
experiment = project.get_experiments(id=experiment_id)[0]
parameters = experiment.get_parameters()
properties = experiment.get_properties()

In [11]:
# parameters

In [12]:
# rcnn transform
transform = GeneralizedRCNNTransform(min_size=int(parameters['MIN_SIZE']),
                                     max_size=int(parameters['MAX_SIZE']),
                                     image_mean=ast.literal_eval(parameters['IMG_MEAN']),
                                     image_std=ast.literal_eval(parameters['IMG_STD']))

In [13]:
# view dataset
# datasetviewer = DatasetViewerSingle(dataset, rccn_transform=None)
# datasetviewer.napari()

In [14]:
# download model from neptune or load from checkpoint
if params['DOWNLOAD']:
    download_path = pathlib.Path(os.getcwd()) / params['DOWNLOAD_PATH']
    download_path.mkdir(parents=True, exist_ok=True)
    model_name = 'best_model.pt'  # that's how I called the best model
    # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
    if not (download_path / model_name).is_file():
        experiment.download_artifact(path=model_name, destination_dir=download_path)  # download model

    model_state_dict = torch.load(download_path / model_name, map_location=torch.device('cuda'))
else:
    checkpoint = torch.load(params['MODEL_DIR'], map_location=torch.device('cuda'))
    model_state_dict = checkpoint['hyper_parameters']['model'].state_dict()

In [15]:
# model init
model = get_faster_rcnn_resnet(num_classes=int(parameters['CLASSES']),
                               backbone_name=ResNetBackbones(parameters['BACKBONE'].split('.')[-1].lower()),  # reverse look-up enum
                               anchor_size=ast.literal_eval(parameters['ANCHOR_SIZE']),
                               aspect_ratios=ast.literal_eval(parameters['ASPECT_RATIOS']),
                               fpn=ast.literal_eval(parameters['FPN']),
                               min_size=int(parameters['MIN_SIZE']),
                               max_size=int(parameters['MAX_SIZE'])
                               )

!!!!!!!!!!anchor_size: 1 cell_anchors: 1


In [16]:
# load weights
model.load_state_dict(model_state_dict)
model.to(torch.device('cuda'))

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(2560,), max_size=2561, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
      

In [19]:
model.eval()
for frame_path in inputs:
    input_video = cv2.VideoCapture(str(frame_path))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    save_dir = pathlib.Path(os.getcwd()) / params['PREDICTIONS_PATH']
    save_dir.mkdir(parents=True, exist_ok=True)
    output_movie = cv2.VideoWriter(os.path.join(save_dir, os.path.basename(frame_path)), fourcc, 30, (2560, 1440)) #(2560, 1440) (1920, 1080)

    while True:
        ret, frame = input_video.read()
        if not ret:
            break
        
        tensor = torch.from_numpy(frame.transpose(2, 0, 1)[np.newaxis, ...]/255).to(device=torch.device('cuda'), dtype=torch.float)
        print(tensor, tensor.size())
        input()
        with torch.no_grad():
            predict = model(tensor)
            # print(predict[0])
            # input()
            if predict[0]['boxes'].numel():
                pred_numpy = [{key: value.cpu().numpy()[0] for key, value in pr.items()} for pr in predict] #predict[0]['boxes'].cpu().numpy()[0]
                if pred_numpy[0]['scores'] > 0.3:
                    box = pred_numpy[0]['boxes']
                    cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0,255,0), 2)

        output_movie.write(frame)

    output_movie.release()
    print('file isOpened:', output_movie.isOpened())

tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.6902, 0.6902, 0.6902],
          [1.0000, 1.0000, 1.0000,  ..., 0.6902, 0.6902, 0.6902],
          [1.0000, 1.0000, 1.0000,  ..., 0.6902, 0.6902, 0.6902],
          ...,
          [0.3608, 0.3608, 0.3608,  ..., 0.0667, 0.0667, 0.0667],
          [0.3647, 0.3647, 0.3647,  ..., 0.0667, 0.0667, 0.0667],
          [0.3647, 0.3647, 0.3647,  ..., 0.0667, 0.0667, 0.0667]],

         [[1.0000, 1.0000, 1.0000,  ..., 0.5333, 0.5333, 0.5333],
          [1.0000, 1.0000, 1.0000,  ..., 0.5333, 0.5333, 0.5333],
          [1.0000, 1.0000, 1.0000,  ..., 0.5333, 0.5333, 0.5333],
          ...,
          [0.3765, 0.3765, 0.3765,  ..., 0.0078, 0.0078, 0.0078],
          [0.3804, 0.3804, 0.3804,  ..., 0.0078, 0.0078, 0.0078],
          [0.3804, 0.3804, 0.3804,  ..., 0.0078, 0.0078, 0.0078]],

         [[1.0000, 1.0000, 1.0000,  ..., 0.3843, 0.3843, 0.3843],
          [1.0000, 1.0000, 1.0000,  ..., 0.3843, 0.3843, 0.3843],
          [1.0000, 1.0000, 1.0000,  ..., 0

KeyboardInterrupt: Interrupted by user

In [20]:
# inference (cpu)
model.eval()
for sample in dataloader_prediction:
    # print("-1-- %s seconds ---" % (time.time() - start_time))
    x, x_names = sample
    x = [t.to(torch.device('cuda')) for t in x]
    with torch.no_grad():
        preds = model(x)
        predicts = [{key: value.cpu().numpy() for key, value in pr.items()} for pr in preds]
        names = [pathlib.Path(x_name) for x_name in x_names]
        save_dir = pathlib.Path(os.getcwd()) / params['PREDICTIONS_PATH']
        save_dir.mkdir(parents=True, exist_ok=True)
        for name, pred in zip(names, predicts):
            pred_list = {key: value.tolist() for key, value in pred.items()}  # numpy arrays are not serializable -> .tolist()
            save_json(pred_list, path=save_dir / name.with_suffix('.json'))

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\iserg\anaconda3\envs\Torch\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\iserg\anaconda3\envs\Torch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\iserg\anaconda3\envs\Torch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\iserg\rebels_code\FasterRCNN\pytorch_faster_rcnn_tutorial\datasets.py", line 171, in __getitem__
    x = self.read_images(input_ID)
  File "c:\Users\iserg\rebels_code\FasterRCNN\pytorch_faster_rcnn_tutorial\datasets.py", line 188, in read_images
    return imread(inp)
  File "c:\Users\iserg\anaconda3\envs\Torch\lib\site-packages\skimage\io\_io.py", line 

In [ ]:
# get prediction files
predictions = get_filenames_of_path(pathlib.Path(os.getcwd()) / params['PREDICTIONS_PATH'])
predictions.sort()

In [22]:
# create prediction dataset
iou_threshold = 0.25 #0.25
score_threshold = 0.45 #0.6

transforms_prediction = ComposeDouble([
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01),
    FunctionWrapperDouble(apply_nms, input=False, target=True, iou_threshold=iou_threshold),
    FunctionWrapperDouble(apply_score_threshold, input=False, target=True, score_threshold=score_threshold)
])

dataset_prediction = ObjectDetectionDataSet(inputs=inputs,
                                            targets=predictions,
                                            transform=transforms_prediction,
                                            use_cache=False)

NameError: name 'predictions' is not defined

In [76]:
# mapping
color_mapping = {
    1: 'green',
}

In [77]:
DatasetViewer(dataset_prediction, color_mapping)

In [78]:
# visualize predictions
datasetviewer_prediction = DatasetViewer(dataset_prediction, color_mapping)
datasetviewer_prediction.napari()
# add text properties gui
datasetviewer_prediction.gui_text_properties(datasetviewer_prediction.shape_layer)

## Experiment with Non-maximum suppression (nms) and score-thresholding

In [138]:
# experiment with nms and score-thresholding
transforms_prediction = ComposeDouble([
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

dataset_prediction = ObjectDetectionDataSet(inputs=inputs,
                                            targets=predictions,
                                            transform=transforms_prediction,
                                            use_cache=False)

color_mapping = {
    1: 'green',
}

datasetviewer_prediction = DatasetViewer(dataset_prediction, color_mapping)
datasetviewer_prediction.napari()

In [112]:
# add score slider
# DOES CURRENTLY NOT WORK
# datasetviewer_prediction.gui_score_slider(datasetviewer_prediction.shape_layer)

In [ ]:
# add nms slider
# DOES CURRENTLY NOT WORK
# datasetviewer_prediction.gui_nms_slider(datasetviewer_prediction.shape_layer)